In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import os
os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [3]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from nb_quantfreedom.nb_simulate import sim_df_backtest
from nb_quantfreedom.sim_order_records import sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [4]:
candles_np = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="5m", candles_to_dl=2000)

It took 00 mins and 01 seconds to download 2000 candles


In [5]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=15,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([1, 3, 6]),
    sl_based_on_add_pct=np.array([0.01, 0.03, .06]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([.1, 0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2, 4]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

In [6]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.logger_pass-215.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.logger_pass-215.py310.1.nbc'
[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.stringer_pass-220.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.stringer_pass-220.py310.1.nbc'
[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_total_ind_settings-96.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_total_ind_settings-96.py310.1.nbc'
Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total orde

,ind_set_idx,dos_index,total_trades,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,54,84.0,-83.65,20.24,0.0,752.751,-836.4573,163.5427
1,0,57,77.0,-84.34,19.48,0.0,769.898,-843.3701,156.6299
2,0,60,65.0,-81.96,16.92,0.0,579.844,-819.6064,180.3936
3,0,72,80.0,-77.09,22.50,0.0,753.019,-770.9107,229.0893
4,0,75,69.0,-70.13,24.64,0.0,860.055,-701.2608,298.7392
...,...,...,...,...,...,...,...,...,...
122,3,435,80.0,-58.84,21.25,0.0,2325.054,-588.3866,411.6134
123,3,450,79.0,-37.37,24.05,0.0,2450.558,-373.7144,626.2856
124,3,453,74.0,-28.17,24.32,0.0,2379.125,-281.7061,718.2939
125,3,468,74.0,-43.78,28.38,0.0,2410.467,-437.7657,562.2343


In [7]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_ind_set_str-114.py310.nbi'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_ind_set_str-114.py310.nbi'
[cache] index saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_ind_set_str-114.py310.nbi'
[cache] data saved to 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_ind_set_str-114.py310.2.nbc'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_evaluate-129.py310.nbi'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_bt_create_ind-53.py310.nbi'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quan

In [8]:
order_records_df.T

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
ind_set_idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
or_set_idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bar_idx,100,101,161,162,163,209,210,212,241,242,...,1814,1822,1823,1831,1832,1837,1838,1848,1849,1860
timestamp,1697684700000,1697685000000,1697703000000,1697703300000,1697703600000,1697717400000,1697717700000,1697718300000,1697727000000,1697727300000,...,1698198900000,1698201300000,1698201600000,1698204000000,1698204300000,1698205800000,1698206100000,1698209100000,1698209400000,1698212700000
datetime,2023-10-19 03:05:00,2023-10-19 03:10:00,2023-10-19 08:10:00,2023-10-19 08:15:00,2023-10-19 08:20:00,2023-10-19 12:10:00,2023-10-19 12:15:00,2023-10-19 12:25:00,2023-10-19 14:50:00,2023-10-19 14:55:00,...,2023-10-25 01:55:00,2023-10-25 02:35:00,2023-10-25 02:40:00,2023-10-25 03:20:00,2023-10-25 03:25:00,2023-10-25 03:50:00,2023-10-25 03:55:00,2023-10-25 04:45:00,2023-10-25 04:50:00,2023-10-25 05:45:00
order_status,EntryFilled,EntryFilled,TakeProfitFilled,EntryFilled,EntryFilled,TakeProfitFilled,EntryFilled,EntryFilled,TakeProfitFilled,EntryFilled,...,EntryFilled,StopLossFilled,EntryFilled,StopLossFilled,EntryFilled,StopLossFilled,EntryFilled,TakeProfitFilled,EntryFilled,StopLossFilled
equity,1000.0,1000.0,949.0638,949.0638,949.0638,898.754,898.754,898.754,850.0556,850.0556,...,21.3522,20.3459,20.3459,19.3452,19.3452,18.363,18.363,17.5886,17.5886,16.478
available_balance,908.478,815.923,949.0638,881.644,813.316,898.754,829.804,752.518,850.0556,800.238,...,19.257,20.3459,17.15,19.3452,14.957,18.363,14.356,17.5886,15.598,16.478
cash_borrowed,7531.085,15066.669,NaN,4820.912,9649.601,NaN,5131.752,10913.03,NaN,2908.57,...,133.813,NaN,268.805,NaN,375.864,NaN,343.237,NaN,120.97,NaN
cash_used,91.522,184.077,NaN,67.42,135.748,NaN,68.95,146.236,NaN,49.818,...,2.095,NaN,3.196,NaN,4.388,NaN,4.007,NaN,1.99,NaN


In [9]:
list(order_records_df.columns)

['ind_set_idx',
 'or_set_idx',
 'bar_idx',
 'timestamp',
 'datetime',
 'order_status',
 'equity',
 'available_balance',
 'cash_borrowed',
 'cash_used',
 'average_entry',
 'fees_paid',
 'leverage',
 'liq_price',
 'possible_loss',
 'total_trades',
 'entry_size_asset',
 'entry_size_usd',
 'entry_price',
 'exit_price',
 'position_size_asset',
 'position_size_usd',
 'realized_pnl',
 'sl_pct',
 'sl_price',
 'tp_pct',
 'tp_price']

In [45]:
temp_df = order_records_df[
    ["order_status", "timestamp", "average_entry", "entry_price", "liq_price", "sl_price", "tp_price"]
]

In [46]:
entries_df = temp_df[temp_df['order_status']=='EntryFilled']

In [48]:
entries_df.values[:,1:].tolist()

[[1697684700000, 28209.4, 28209.4, 28121.0, 28149.1, 28354.3],
 [1697685000000, 28208.9, 28208.0, 28120.9, 28149.1, 28353.1],
 [1697703300000, 28335.5, 28335.5, 28195.9, 28224.1, 28531.9],
 [1697703600000, 28334.7, 28333.0, 28195.9, 28224.1, 28529.8],
 [1697717700000, 28485.9, 28485.9, 28360.8, 28389.2, 28668.0],
 [1697718300000, 28479.6, 28467.7, 28360.8, 28389.2, 28652.0],
 [1697727300000, 28621.0, 28621.0, 28392.4, 28420.8, 28907.0],
 [1697732400000, 28821.7, 28821.7, 28523.8, 28552.3, 29177.4],
 [1697732700000, 28832.0, 28852.9, 28533.6, 28562.1, 29193.2],
 [1697737200000, 28528.7, 28528.7, 28452.6, 28509.9, 28633.1],
 [1697737500000, 28481.9, 28481.9, 28405.9, 28436.0, 28613.2],
 [1697739600000, 28609.3, 28609.3, 28407.6, 28436.0, 28868.3],
 [1697739900000, 28608.9, 28608.1, 28407.6, 28436.0, 28867.3],
 [1697748300000, 28816.7, 28816.7, 28638.3, 28667.0, 29052.8],
 [1697748600000, 28804.6, 28782.4, 28638.3, 28667.0, 29021.9],
 [1697749500000, 28707.9, 28707.9, 28620.5, 28649.2, 28

In [60]:
np.reshape(candles_np[:20,0].astype(np.int64), (-1,20)).tolist()

[[1697654700000,
  1697655000000,
  1697655300000,
  1697655600000,
  1697655900000,
  1697656200000,
  1697656500000,
  1697656800000,
  1697657100000,
  1697657400000,
  1697657700000,
  1697658000000,
  1697658300000,
  1697658600000,
  1697658900000,
  1697659200000,
  1697659500000,
  1697659800000,
  1697660100000,
  1697660400000]]